In [29]:
import tensorflow as tf
import tensorflow_probability as tfp
tf_flaot = tf.float64

In [30]:
import os

In [32]:
def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t

In [56]:
def clean_text(txt):
    try:
        #txt = re.search(r'<text.*?>(.*)</text', txt, flags=re.DOTtxt).group(1)
        txt = re.sub(r'\n', ' ', txt)
        txt = re.sub(r'\{\{.*?\}\}', r'', txt)
        txt = re.sub(r'\[\[Category:.*', '', txt)
        txt = re.sub(r''
        txt = re.sub(r'==\s*[Ss]ource\s*==.*', '', txt)
        txt = re.sub(r'==\s*[Rr]eferences\s*==.*', '', txt)
        txt = re.sub(r'==\s*[Ee]xternal [Ll]inks\s*==.*', '', txt)
        txt = re.sub(r'==\s*[Ee]xternal [Ll]inks and [Rr]eferences==\s*', '', txt)
        txt = re.sub(r'==\s*[Ss]ee [Aa]lso\s*==.*', '', txt)
        txt = re.sub(r'http://[^\s]*', '', txt)
        txt = re.sub(r'\[\[Image:.*?\]\]', '', txt)
        txt = re.sub(r'Image:.*?\|', '', txt)
        txt = re.sub(r'\[\[.*?\|*([^\|]*?)\]\]', r'\1', txt)
        txt = re.sub(r'\&lt;.*?&gt;', '', txt)
        return txt
    except:
        # Something went wrong, try again. (This is bad coding practice.)
        print("oops. there was a failure parsing.")

In [57]:
k=0

for event, elem in etree.iterparse("./xml.xml", events=('start', 'end')):
    if event=="start":
        continue
    tname = strip_tag_name(elem.tag)
    if tname=="text":
#        print(elem.text)
        print(clean_text(elem.text))
    if tname=="redirect":
        print(elem.attrib['title'])
    k+=1
    if k>1000:
        break

Computer accessibility
#REDIRECT Computer accessibility    
        '''Anarchism''' is an anti-authoritarian political and social philosophy that rejects hierarchy as unjust and advocates its replacement with self-managed, self-governed societies based on voluntary, cooperative institutions. These institutions are often described as stateless societies, although several authors have defined them more specifically as distinct institutions based on non-hierarchical or free associations. Anarchism's central disagreement with other ideologies is that it holds the state to be undesirable, unnecessary, and harmful.  Anarchism is usually placed on the far-left of the political spectrum, and much of its economics and legal philosophy reflect anti-authoritarian interpretations of communism, collectivism, syndicalism, mutualism, or participatory economics. As anarchism does not offer a fixed body of doctrine from a single particular worldview, many anarchist types and traditions exist and variet

In [10]:
a = tf.Variable([1,2,3], dtype=tf_flaot)

a.assign([2,3,4])

tf.gather_nd(
    a, [[1],[2]], batch_dims=0, name=None
)

a[1:2]

tf.sparse.

loggamma = tf.math.lgamma(a)

grad = tf.gradients(loggamma, a)



In [11]:
dt_string

'20200302082048'